- Purpose: 
To calculate one's grade in Linear Algebra by simply copy and pasting the raw information form D2L (where our grades are tracked).

- Setup:
Copy and paste the data into ChatGPT and give it this prompt:

"Put this into a csv.
The names of the columns should be: GRADE_ITME, POINTS.
The column for POINTS (i.e. 3 / 3) should be exactly (3 / 3), not 1, or otherwise simplified.
Delete rows that do not contain any numbers."

Imports

In [42]:
# %pip install numpy
# %pip install pandas
import numpy as np
import pandas as pd

Read in and format data

In [35]:

# Read in CSV
df = pd.read_csv('DaltonGrades.csv')

# rename GRADE_ITEMs for ease of use
df['GRADE_ITEM'] = df['GRADE_ITEM'].str.replace(' ', '_').str.upper()
df['GRADE_ITEM'] = df['GRADE_ITEM'].str.replace('-', '_').str.upper()

Derive columns necessary for calculation

In [36]:
# seperate "POINTS" into "POINTS_EARNED" and "POINTS_POSSIBLE" 
df[['POINTS_EARNED', 'POINTS_POSSIBLE']] = df['POINTS'].str.split('/', expand=True)

# create column ITEM_TYPE based off of GRADE_ITEM being either IN_CLASS, HOMEWORK, or EXAM
df['ITEM_TYPE'] = np.where(df['GRADE_ITEM'].str.contains('IN_CLASS'), 'IN_CLASS', 
                           np.where(df['GRADE_ITEM'].str.contains('HOMEWORK'), 'HOMEWORK', 
                                    np.where(df['GRADE_ITEM'].str.contains('EXAM'), 'EXAM', 'HOMEWORK')))

# compute grade via points
df['GRADE'] = (df['POINTS_EARNED'].astype(float) / df['POINTS_POSSIBLE'].astype(float)) * 100

# sort for visual purposes
df.sort_values(by=['ITEM_TYPE', 'GRADE_ITEM'], inplace=True)

# print df for sake of checking
df

,GRADE_ITEM,POINTS,POINTS_EARNED,POINTS_POSSIBLE,ITEM_TYPE,GRADE
9,EXAM_1,64.5 / 86,64.5,86,EXAM,75.000000
18,EXAM_2,55 / 89,55,89,EXAM,61.797753
25,FINAL_PROPOSAL,12 / 12,12,12,HOMEWORK,100.000000
17,LAB_1,0 / 10,0,10,HOMEWORK,0.000000
0,WEBWORK,80 / 100,80,100,HOMEWORK,80.000000
4,WRITTEN_HOMEWORK_#1,10 / 13,10,13,HOMEWORK,76.923077
8,WRITTEN_HOMEWORK_#2,10 / 12,10,12,HOMEWORK,83.333333
14,WRITTEN_HOMEWORK_#3,6 / 11,6,11,HOMEWORK,54.545455
23,WRITTEN_HOMEWORK_#4,16 / 21,16,21,HOMEWORK,76.190476
30,WRITTEN_HOMEWORK_#5,17 / 22,17,22,HOMEWORK,77.272727


Calculate grade for In Class assignments

In [37]:
# filter down to just in class
in_class_df = df[df['ITEM_TYPE'] == 'IN_CLASS'].sort_values(by='GRADE', ascending=False)

# select all but the 3 lowest grades (professor drops 3 lowest grades)
in_class_df = in_class_df.iloc[:-3]

# compute in class grade
in_class_earned = in_class_df['POINTS_EARNED'].astype(float).sum()
in_class_possible = in_class_df['POINTS_POSSIBLE'].astype(float).sum()
in_class_grade = (in_class_earned / in_class_possible) * 100

Calculate grade for Homework assignements

In [38]:
# filter down to just hw
hw_df = df[df['ITEM_TYPE'] == 'HOMEWORK'].sort_values(by='GRADE', ascending=False)

# calculate hw grade
hw_possible = hw_df['POINTS_POSSIBLE'].astype(float).sum()
hw_earned = hw_df['POINTS_EARNED'].astype(float).sum()
hw_grade = (hw_earned / hw_possible) * 100

Calculate grade for Exams

In [39]:
# filter to exams
exam_df = df[df['ITEM_TYPE'] == 'EXAM'].sort_values(by='GRADE', ascending=False)

# calculate exam grade
exam_possible = exam_df['POINTS_POSSIBLE'].astype(float).sum()
exam_earned = exam_df['POINTS_EARNED'].astype(float).sum()
exam_grade = (exam_earned / exam_possible) * 100

Calculate total grade

In [40]:
# calculate total_grae
total_grade = ((in_class_earned + hw_earned + exam_earned) / (in_class_possible + hw_possible + exam_possible)) * 100

Print results

In [41]:
# print results
print('In Class Grade: ' + str(in_class_grade))
print('Homework Grade: ' + str(hw_grade))
print('Exam Grade: ' + str(exam_grade))
print('Total Grade: ' + str(total_grade))

In Class Grade: 93.50961538461539
Homework Grade: 75.12437810945273
Exam Grade: 68.28571428571428
Total Grade: 76.61458333333333
